# 图片的几何变换

- 计算机高级视觉处理的基础，对图片数据的矩阵操作
    - 实质：矩阵操作
    - 给矩阵不同的系数，完成不同的运算符方式
    
- 图片缩放
    - 改变图片的高宽，放大/缩小
    
- 图片剪切
    - 截取矩阵中的某一块
    
- 图片位移
    - 移动图片的x,y
    
- 图片镜像
    - 创建一个更大的矩阵，将原图倒序绘制一次
    
- 图片仿射变换：
    - 位移、旋转、缩放  

In [1]:
import cv2
import numpy as np
img = cv2.imread("./eunha.jpg")

height = img.shape[0]
width = img.shape[1]
channel = img.shape[2]

# 1. 图片缩放


- 4种常见图片缩放方法：
    - 最近临域插值
    - 双线性插值
    - 像素关系重采样
    - 立方插值
    
- API：resize

- 算法原理
    - 最近临域插值
    - 双线性插值
    
- 源码
    - 最近临域插值

In [3]:
# 放大/缩小 ; 等比例/非等比例
dstHeight = int(height*0.5)
dstWidth = int(width*0.5)


# 双线性插值：
dst = cv2.resize(img,(dstWidth,dstHeight))

cv2.imshow("img", img)
cv2.imshow("resize",dst)
cv2.waitKey(10000)
cv2.destroyAllWindows() 

## 1.1 最近临域插值原理：

- 原图像scr的维度(10x20)，目标图像dst的维度(5x10)

- 公式：
    - src_x = dst_x * (scr行数 / dst行数)
    - src_y = dst_y * (scr列数 / dst列数)

- 例：计算目标图像点dst(1,2)对应原图像scr的点(x,y)
    - src_x = dst_x * (scr行数 / dst行数) = 1 * (10/5) = 2
    - src_y = dst_y * (scr列数 / dst列数) = 2 * (20/10) = 4
    - 目标图像点(1,2)对应原图像像素点(2,4)
    - 若计算结果为12.3，则取最近值12，最近临域插值法
    

## 1.2 双线性插值原理：

- 目标点为(15.2,22.3)
- 在(15,22), (15,23), (16,22), (16,23)围成的矩形
- 使用权重表示目标点零件

In [4]:
# 使用源码完成最近领域插值法：

dstImage = np.zeros([dstHeight, dstWidth, 3], np.uint8)    # 创建空白模板
for i in range(dstHeight):
    for j in range(dstWidth):
        iNew = int(i*(height*1.0/dstHeight))    # 转换数据类型：乘以1.0
        jNew = int(j*(width*1.0/dstWidth))
        dstImage[i,j] = img[iNew,jNew]

cv2.imshow("img", img)
cv2.imshow("dst",dstImage)
cv2.waitKey(10000)
cv2.destroyAllWindows() 

# 2. 图片剪切

In [5]:
# x轴：保留100 -> 200
# y轴：保留100 -> 300

dst = img[100:200, 100:300]

cv2.imshow("img", img)
cv2.imshow("image", dst)
cv2.waitKey(10000)
cv2.destroyAllWindows() 

# 3. 图片镜像

1. 创建一个足够大的画板
    - 宽度一致，高度为原图的两倍
2. 将一副图像分别从前向后，从后向前绘制
3. 绘制中心分割线

In [8]:
mirror_img = np.zeros((height*2, width, channel), np.uint8)

for y in range(height):
    for x in range(width):
        mirror_img[y,x] = img[y,x]                # 上半部分：从前往后
        mirror_img[height*2 - y - 1, x] = img[y,x]    # 下半部分，从后往前

for i in range(width):
    mirror_img[height,i] = (0,0,255) #bgr
    

cv2.imshow("img", img)
cv2.imshow("mirror", mirror_img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

# 4. 仿射变换：

- 原理
    - 仿射变换是指，在向量空间中进行一次线性变换(乘以一个矩阵)并加上一个平移(加上一个向量)，变换为另一个向量空间的过程。

- 例：
    - 把原图片上的三个像素点，映射到目标图片的三个新的位置上。
        - 三个位置：左上角，左下角，右上角
        - src 3点 --> dst 3点

- 放射变换的应用
    - 4.1 位移
    - 4.2 缩放
    - 4.3 旋转

In [2]:
matSrc = np.float32([[0,0],[0,height-1],[width-1,0]])             # 原图片上的3个点：左上，左下，右上
matDst = np.float32([[50,50],[300,height-200 ],[width-300,100]])    # 映射到目标图片的3个点

# 组合矩阵
matAffine = cv2.getAffineTransform(matSrc, matDst)    # 获取放射变换矩阵：参数1：原图像的三个点，参数2：目标图像的三个点

# 使用仿射变换，显示图片
dst = cv2.warpAffine(img, matAffine, (width,height))

cv2.imshow("img", img)
cv2.imshow('dst',dst)
cv2.waitKey(10000)
cv2.destroyAllWindows()

## 4.1 图片移位


- API，算法原理，源代码


- 算法原理：
    - [[A1,A2,B1],[A3,A4,B2]]
        - 拆分：[[A1,A2],[A3,A4]],[[B1],[B2]]
        - 输入:[[x],[y]]
        - newX = A1 * x + A2 * y + B1
        - newY = A3 * x + A4 * y + B2
        
    - 将2x3的矩阵：[[1,0,100],[0,1,200]]，拆分成A和B
        - A = [[1,0],[0,1]]  2x2
        - B = [[100],[200]]  2x1
        - 输入的xy为C，
            - C = [[x],[y]]  2x1
        - 计算公式：
            - 当前移位之后的矩阵: A x C + B = [[1,0],[0,1]] * [[x],[y]] + [[100],[200]] 
                                         = [[x],[y]] + [[100],[200]]
                                         = [[x+100],[y+200]]

In [6]:
# 设置一个移位矩阵
matShift = np.float32([[1,0,100],[0,1,200]])    # 2x3，沿水平方向移动100个像素，竖直方向移动200个像素
dst = cv2.warpAffine(img, matShift,(width, height))
# 仿射方法：参数1：原数据，参数2：位移矩阵，参数3：原数据形状（x轴width，y轴height）

cv2.imshow("img", img)
cv2.imshow("dst", dst)
cv2.waitKey(10000)
cv2.destroyAllWindows() 

In [7]:
# 通过像素实现移动（源码）
# 将图片每一个点的坐标xy，经过映射之后，投影到新的xy坐标
# (10,20) --> (210,120)

# 定义空矩阵
dst = np.zeros(img.shape, np.uint8)

for i in range(height-200):           # 让图片向下移200个像素
    for j in range(width-100):    # 让图片向右移100个像素，原图最后100个像素不显示
        dst[i+200,j+100] = img[i,j]

cv2.imshow("img", img)
cv2.imshow("dst", dst)
cv2.waitKey(10000)
cv2.destroyAllWindows() 

## 4.2 图片缩放

- 要求
    - x -> 0.5x
    - y -> 0.5y
- 方法：
    - [[0.5,0,0],[0,0.5,0]]
    - 拆分：[[0.5, 0],[0, 0.5]]和[[0],[0]]
    - 输入:[[x],[y]]
    - newX = 0.5 * x + 0 * y + 0
    - newY = 0 * x + 0.5 * y + 0

In [ ]:
# 定义一个缩放矩阵
mat_scale = np.float32([[0.5,0,0],[0,0.5,0]])
dst = cv2.warpAffine(img, mat_scale,(int(width/2), int(height/2)))

cv2.imshow("img", img)
cv2.imshow("small", dst)
cv2.waitKey(10000)
cv2.destroyAllWindows()

## 4.3 图片旋转

In [7]:
# 获取旋转矩阵：
matRotation = cv2.getRotationMatrix2D((width*0.5,height*0.5), 45, 0.5)
# matRotation = cv2.getRotationMatrix2D((height*0.5, width*0.5), 45, 0.5)

# 参数1：旋转的中心点
# 参数2：旋转的角度
# 参数3：缩放的系数

# 调用仿射方法，完成仿射变换：
dst = cv2.warpAffine(img, matRotation,(width, height))


cv2.imshow("img", img)
cv2.imshow('dst',dst)
cv2.waitKey(10000)
cv2.destroyAllWindows()